# What metrics I am using, why and how to improve.

**Goal of notebook**:  take one example text that has text generated, at 70%, and try to understand each of the metrics one by one

## 1. Setup and Configuration

In [19]:
import sys
import os
import pandas as pd
from pathlib import Path

# go to project root
project_root = Path(os.getcwd()).parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [20]:
from configs.experiment_config import baseline, extended

# old approach
# config_qwen = experiment_config.EXPERIMENT_BASELINE_ONLY_SONGS_QWEN
config_qwen = extended(
	model="qwen2.5:0.5b-instruct", 
	context_delay_seconds=3.0,
)
config_qwen.start_logging()

INFO:configs.experiment_config:Running experiment: memorisation_extended
INFO:configs.experiment_config:Contexted to run: [5, 25, 50, 75, 90]


In [21]:
from nudging.models import OllamaClient

# initialise the client
client_qwen = OllamaClient(
	model=config_qwen.model_config.name,
	max_tokens=config_qwen.model_config.max_tokens
	)

In [22]:
from nudging.data_loader import load_data

# TODO: clean this so i am not writing all this code for loading data.
# load data
dataset = load_data(
    base_dir=project_root / config_qwen.data_config.data_folder_name,
    min_words=config_qwen.data_config.min_word_count,
    max_samples=config_qwen.max_samples,
    categories=config_qwen.data_config.categories
)
print(f"loaded the data: {len(dataset)} files.")

INFO:nudging.data_loader:Starting data load from: /Users/abditimer/Documents/PhD/experiments/nudging/data
INFO:nudging.data_loader:Scanning directory: /Users/abditimer/Documents/PhD/experiments/nudging/data
INFO:nudging.data_loader:Skipping non-file: /Users/abditimer/Documents/PhD/experiments/nudging/data/songs
INFO:nudging.data_loader:Skipping non-file: /Users/abditimer/Documents/PhD/experiments/nudging/data/podcasts
INFO:nudging.data_loader:Skipping non-file: /Users/abditimer/Documents/PhD/experiments/nudging/data/songs/taylor_swift
INFO:nudging.data_loader:Skipping non-file: /Users/abditimer/Documents/PhD/experiments/nudging/data/podcasts/huberman
INFO:nudging.data_loader:Kept songs::taylor_swift::the_fate_of_ophelia: 432 words
INFO:nudging.data_loader:Kept songs::taylor_swift::shake_it_off: 560 words
INFO:nudging.data_loader:Loaded 2 files
INFO:nudging.data_loader:Load complete.


loaded the data: 2 files.


## 2. Limit to only one song for this experiment. 

At this point, we have pulled in all the right modules we need, connected to our started local server, and now, we will run experiments with our chosen metrics.

In [23]:
del dataset['songs::taylor_swift::shake_it_off']

In [24]:
dataset

{'songs::taylor_swift::the_fate_of_ophelia': "I heard you calling\nOn the megaphone\nYou wanna see me all alone\nAs legend has it you\nAre quite the pyro\nYou light the match to watch it blow\nAnd if you'd never come for me\nI might've drowned in the melancholy\nI swore my loyalty to me, myself and I\nRight before you lit my sky up\nAll that time\nI sat alone in my tower\nYou were just honing your powers\nNow I can see it all (see it all)\nLate one night\nYou dug me out of my grave and\nSaved my heart from the fate of\nOphelia\nKeep it one hundred\nOn the land, the sea, the sky\nPledge allegiance to your hands\nYour team, your vibes\nDon't care where the hell you been\n'Cause now you're mine\nIt's 'bout to be the sleepless night\nYou've been dreaming of\nThe fate of Ophelia\nThe eldest daughter of a nobleman\nOphelia lived in fantasy\nBut love was a cold bed full of scorpions\nThe venom stole her sanity\nAnd if you'd never come for me\nI might've lingered in purgatory\nYou wrap around 

This is what happens in my code:
1. load the data
2. we call `run_experiment` in `run_memorisation_experiment`
3. this then calls `run_experiments` in `nudging.experiment` - sidenote: this is confusing!
4. this calculates the metrics on the fly once the text has been generated.

## Deep dive into my code

Data includes:
1. title: This is the label. It has <data_type>::<data_owner>::<data_name> format
2. content: the actual content 

In [25]:
title, content = next(iter(dataset.items()))

for this notebook, we are breaking it down. We will look at everything so we can build the best experiment. Facts.

In [26]:
title

'songs::taylor_swift::the_fate_of_ophelia'

In [27]:
content

"I heard you calling\nOn the megaphone\nYou wanna see me all alone\nAs legend has it you\nAre quite the pyro\nYou light the match to watch it blow\nAnd if you'd never come for me\nI might've drowned in the melancholy\nI swore my loyalty to me, myself and I\nRight before you lit my sky up\nAll that time\nI sat alone in my tower\nYou were just honing your powers\nNow I can see it all (see it all)\nLate one night\nYou dug me out of my grave and\nSaved my heart from the fate of\nOphelia\nKeep it one hundred\nOn the land, the sea, the sky\nPledge allegiance to your hands\nYour team, your vibes\nDon't care where the hell you been\n'Cause now you're mine\nIt's 'bout to be the sleepless night\nYou've been dreaming of\nThe fate of Ophelia\nThe eldest daughter of a nobleman\nOphelia lived in fantasy\nBut love was a cold bed full of scorpions\nThe venom stole her sanity\nAnd if you'd never come for me\nI might've lingered in purgatory\nYou wrap around me like a chain, a crown, a vine\nPulling me 

For this experiment, we were originally cycling through different percentages. We will now only focus on one singular experiment.

In [28]:
config_qwen.context_percentages

[5, 25, 50, 75, 90]

Lets focus only on 90%. 

What does this mean? It means, for our experiment, we want to take 90% of that text and predict the remaining 10%.

In [29]:
current_context_perct = config_qwen.context_percentages[3]
current_context_perct

75

## Peeking inside run_memorisation_experiment

We will now peek into how our module builds out this experiments. We will build it out one step at a time.

We start by passing the content to 'run_memorisation_experiment' -> 'experiment' ->.

In 'experiment', we have a '_get_split_text() function that does the following:

In [30]:
"""Split text into test portion and remaining portion"""
from collections import defaultdict
d = defaultdict(str)
# .split() methods divides a string into a list of substrings split by seperator (' ')
words = content.split()
current_context_wordcount = int(len(words) * (current_context_perct / 100))
d['train_words'] = " ".join(words[:current_context_wordcount])
d['test_words'] = " ".join(words[current_context_wordcount:])

In [31]:
words[:5]

['I', 'heard', 'you', 'calling', 'On']

content: one long string of the full text
words = array of word strings

In [32]:
total_words = len(words)
train_words_count = len(d['train_words'].split())
test_words_count = len(d['test_words'].split())

In [33]:
print(f"""Total words: {total_words}
experiment context split: {current_context_perct}%

Count of train words: {train_words_count}
Count of test words: {test_words_count}

Hence:
count of train words + count of test words = total words 
{train_words_count} + {test_words_count} = {total_words}
""")


Total words: 432
experiment context split: 75%

Count of train words: 324
Count of test words: 108

Hence:
count of train words + count of test words = total words 
324 + 108 = 432



Then we call _generate_response

In [34]:
# Create prompt
prompt = f"""
Generate characters when you see <Generate>
You must generate exactly {test_words_count} words!
Continue the text that comes after <StartText>. 
<StartText>
{d['train_words']}
</StartText>
<Generate>"""

In [35]:
# Generate with model
generated_response = client_qwen.generate(prompt=prompt)

In [36]:
generated_response

"I heard you calling On the megaphone You wanna see me all alone As legend has it you Are quite the pyro You light the match to watch it blow And if you'd never come for me I might've drowned in the melancholy I swore my loyalty to me, myself and I Right before you lit my sky up All that time I sat alone in my tower You were just honing your powers Now I can see it all (see it all) Late one night You dug me out of my grave and Saved my heart from the fate of Ophelia Keep it one hundred On the land, the sea, the sky Pledge allegiance to your hands Your team, your vibes Don't care where the hell you been 'Cause now you're mine It's about to be the sleepless night You've been dreaming of The fate of Ophelia The eldest daughter of a nobleman Ophelia lived in fantasy But love was a cold bed full of scorpions The venom stole her sanity And if you'd never come for me I might've lingered in purgatory You wrap around me like a chain, a crown, a vine Pulling me into the fire All that time I sat 

As you can see, our generation has 2 problems currently:
1. we need to control the stopping criteria in our models.py (in other words, when we are reaching out to our model via ollama)
2. have a stopping criteria to trim - post-processing.

### 1. Stopping at the model call level
pass `max_tokens` to OllamaClient to restrict how many tokens are generated. A simple heuristic to implement:
- max_token_cap = ceiling(test_words_count * 1.15) ~ 15%

In [37]:
# temp for now
import logging
logging.getLogger().setLevel(logging.DEBUG)

In [38]:
config_qwen_updated = extended(
	model="qwen2.5:0.5b-instruct", 
	context_delay_seconds=3.0,
	max_tokens=test_words_count
)

config_qwen_updated.start_logging()

client_qwen_updated = OllamaClient(
	model=config_qwen_updated.model_config.name,
	max_tokens=config_qwen_updated.model_config.max_tokens
)

INFO:configs.experiment_config:Running experiment: memorisation_extended
INFO:configs.experiment_config:Contexted to run: [5, 25, 50, 75, 90]


We can keep everything the same, all variables etc. We only change how the endpoint model is being called by passing the `max_tokens` param to the model.

In [39]:
generated_response_limited = client_qwen_updated.generate(prompt=prompt)

DEBUG:nudging.models:max_tokens=108
                num_predict=162
                multiplier=1.5
            
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/generate HTTP/1.1" 200 None


In [40]:
print(generated_response_limited)

I heard you calling on the megaphone, "You wanna see me all alone As legend has it you Are quite the pyro You light the match to watch it blow And if you'd never come for me I might’ve drowned in the melancholy I swore my loyalty to me, myself and I Right before you lit my sky up All that time I sat alone in my tower You were just honing your powers Now I can see it all (see it all) Late one night You dug me out of my grave and Saved my heart from the fate of Ophelia Keep it one hundred On the land, the sea, the sky Pledge allegiance to your hands Your team, your vibes Don't care where the hell you been 'Cause now you're mine It's 'bout to be the sleep


Now lets compare the 2 generated outputs lengths.

In [41]:
print(f"Expected length of generated words: {test_words_count}")

print(f"No limiting length: {len(generated_response.split())}")
print(f"Limited length generation: {len(generated_response_limited.split())}")

Expected length of generated words: 108
No limiting length: 324
Limited length generation: 139


### 2. Stopping post-process

In [42]:
def trim_to_n_words(text: str, max_tokens: int) -> str:
	words = text.strip().split()
	return " ".join(words[:max_tokens])

In [43]:
trimmed_response = trim_to_n_words(generated_response, test_words_count)

In [44]:
trimmed_response

"I heard you calling On the megaphone You wanna see me all alone As legend has it you Are quite the pyro You light the match to watch it blow And if you'd never come for me I might've drowned in the melancholy I swore my loyalty to me, myself and I Right before you lit my sky up All that time I sat alone in my tower You were just honing your powers Now I can see it all (see it all) Late one night You dug me out of my grave and Saved my heart from the fate of Ophelia Keep it one hundred On the"

# Now lets dive into the metrics!

We will be looking deeply at my 4 original metrics, and then expanding them to include new ones. Think of this part of the notebook as now analysing the metrics